# Testing Google Generative AI Embeddings

This notebook tests the proper initialization of GoogleGenerativeAIEmbeddings.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
import os

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# Correct initialization using 'model' parameter
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

In [ ]:
# Test the embeddings with a simple text
text = "This is a test sentence for embeddings."
embedding = embeddings.embed_query(text)

# Print the length of the embedding vector
print(f"Embedding dimension: {len(embedding)}")

## Testing Simple RAG Example

Now let's test a simple RAG application using the correct embeddings.

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain

# Create sample documents
from langchain_core.documents import Document
docs = [
    Document(page_content="LangChain is a framework for developing applications powered by language models.", metadata={"source": "docs"}),
    Document(page_content="RAG stands for Retrieval Augmented Generation.", metadata={"source": "docs"}),
    Document(page_content="Embeddings are vector representations of text.", metadata={"source": "docs"})
]

In [ ]:
# Create the vector store
vector_store = FAISS.from_documents(docs, embeddings)

# Initialize LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    google_api_key=os.getenv("GOOGLE_API_KEY")
)

# Create the prompt
prompt = ChatPromptTemplate.from_template(
    """Answer the following question based on the context below.
    <context>
    {context}
    </context>
    
    Question: {input}
    If the question is not answerable based on the context, say "I don't know"."""
)

# Create the document chain
document_chain = create_stuff_documents_chain(llm, prompt)

# Create retrieval chain
retriever = vector_store.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
# Test the retrieval chain
response = retrieval_chain.invoke({"input": "What is LangChain?"})
print(response["answer"])